In [91]:
### This script re-annotate covariate file
# Author: Yiyun

In [1]:
import pandas as pd
import os, pickle, mmap
from multiprocessing import Pool

***
**re-annotate covariate file**

In [1]:
import math,pickle,os
import pandas as pd

In [2]:
### Read files
dir_ref = '../data/proc_refs'
dir_mutsig_in = '../data/MutSigCVsyn_inputs'

lgene = pickle.load(open(os.path.join(dir_ref,'gene_name_list_102121.pkl'),'rb'))
df_covar = pd.read_csv(os.path.join(dir_mutsig_in, 'gene.covariates.txt'),sep = '\t',index_col = 0)
df_covar_ext = pd.read_csv(os.path.join(dir_ref,'./gene.name.external.txt'), sep = '\t')
df_covar_ext = df_covar_ext.dropna()

In [4]:
### Drop na, drop duplicated genes in df_covar_ext
#1. drop if the synonyms is the same with original name
drop_idx = df_covar_ext[df_covar_ext.apply(pd.Series.nunique, axis=1) == 1].index.tolist()
idx_all = df_covar_ext.index.tolist()
keep_idx = [i for i in idx_all if i not in drop_idx]
df_covar_ext = df_covar_ext.loc[keep_idx,:]
# Drop if the synonym names are the same
df_covar_ext = df_covar_ext.drop_duplicates('synonym')
df_covar_ext = df_covar_ext.set_index('original')

In [7]:

lsub_name_all = {}
for genes in lgene:
    if genes not in df_covar.index:   # If the gene is not in covariate file
        print(genes)
        # find the gene in the external dataframe
        try:
            ext_names = df_covar_ext.loc[genes,'synonym']
            lsub_name = []
            # If the synonym name is in covariate file
            if isinstance(ext_names, str):
                if ext_names in df_covar.index.tolist():
                    lsub_name.append(ext_names)
            elif len(ext_names)> 0:
                for name in ext_names:
                    if name in df_covar.index.tolist():
                        lsub_name.append(name)
            lsub_name_all[genes] = lsub_name
        except KeyError:
            pass # some genes is not in the ext file

C10orf40
AL355149.2
EFCAB11
GLTSCR1L
KIAA0040
ZNF729
UNC80
AC096677.1
TONSL
GGNBP1
BOD1L1
SRSF11
NOXRED1
C6orf132
FAM196B
IFITM10
RWDD4
EEF1E1-BLOC1S5
AGO4
METTL18
RP11-1220K2.2
MCMDC2
SEC14L6
EIF2D
TRAPPC2P1
TMEM247
TRMT10C
SPIDR
CAPN14
SCGB2B2
ZKSCAN8
MGC4294
MEF2BNB
PRADC1
ZNF66
CCZ1B
CCDC176
SMCO4
PET112
FKSG63
TEX28P1
AP001652.1
AC018755.1
CLDN24
ACKR3
MCIDAS
SPATA3
ENKD1
TRIM51
VWDE
AC016559.1
TRMT10A
SPTSSA
VSTM5
SPECC1L
UQCC2
RP11-89N17.1
TEX40
CTSL
DEFB4B
DAZ3
RTFDC1
CCDC169-SOHLH2
HIST2H3A
ADIRF
APITD1-CORT
BPIFB1
ZNF716
SIGLECL1
ORC2
FLJ20306
RP11-795F19.5
PPP1R3E
KANSL3
GAGE12C
KMT2D
CNPPD1
AL591479.1
C22orf46
FAM228B
AGAP6
CHDC2
MYRFL
GSKIP
RP11-458D21.5
C22orf26
MZT1
GOLGA8R
C2CD4D
AMER2
COA1
SH3D21
EFR3B
TMEM238
SUPT20H
C22orf34
FDCSP
SPRYD7
M1AP
TMEM232
NELFB
LGALS16
SDIM1
C21orf54
AC006455.1
AL137798.1
DNAAF3
AGMO
RP11-637O19.3
MBD3L3
HDHD1
AREGB
ZNF865
LAMTOR5
AGAP5
IL36A
FUOM
SMIM11
CTD-2144E22.5
ZNF879
AL450307.1
ARHGEF40
IL36RN
FLJ30594
GDF5OS
GTSCR1
RP11-146D12.2


In [10]:
'SLAIN2' in lsub_name_all

False

In [6]:
n

2043

In [17]:
for k,v in lsub_name_all.copy().items():
    # if 2 synonymous name is in the covariate file, means that one of them is not the gene we are referring to. 
    if len(v)>1: del lsub_name_all[k]
    # delete the one that is empty
    if len(v)==0: del lsub_name_all[k]

In [18]:
dict_sub = {v[0]: k for k, v in lsub_name_all.items()}

In [19]:
len(dict_sub) # number of genes to convert

862

In [20]:
df_covar = df_covar.rename(index=dict_sub)

In [21]:
# df_covar.to_csv(os.path.join(dir_mutsig_in,'gene.covariates.new.txt'), sep = '\t')